In [1]:
! export CUDA_LAUNCH_BLOCKING=1

In [2]:
import pandas as pd
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from tqdm.autonotebook import tqdm

tqdm.pandas()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
checkpoint_bloomz_1b7 = "bigscience/bloomz-1b7"

tokenizer_bloomz_1b7 = AutoTokenizer.from_pretrained(checkpoint_bloomz_1b7)
model_bloomz_1b7 = AutoModelForCausalLM.from_pretrained(checkpoint_bloomz_1b7, torch_dtype="auto", device_map="auto")

In [4]:
# prompt = (
#     "Generate story using following story idea: \n\n"
#     "IDEA: \nYou've finally managed to discover the secret to immortality. Suddenly, Death appears before you, hands you a business card, and says, `` When you realize living forever sucks, call this number, I've got a job offer for you. ''\n"
#     "STORY: \n"
# )

# inputs = tokenizer_bloomz_1b7.encode(prompt, return_tensors="pt").to("cuda")
# outputs = model_bloomz_1b7.generate(inputs, do_sample=True, min_length=100, max_new_tokens=500)
# print(tokenizer_bloomz_1b7.decode(outputs[0]))

In [21]:
def generate_stories(model, tokenizer, prompts, do_sample=True, min_length=30, max_new_tokens=100):
    generated_stories = []
    for prompt in tqdm(prompts):
        inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(inputs, 
                                 do_sample=do_sample, 
                                 min_length=min_length, 
                                 max_new_tokens=max_new_tokens,
                                 top_k=50,
                                 top_p=0.9)
        generated_stories.append(tokenizer.decode(outputs[0]))
    return generated_stories

In [6]:
# def batch(iterable, n=1):
#     l = len(iterable)
#     for ndx in range(0, l, n):
#         yield iterable[ndx:min(ndx + n, l)]

In [7]:
# def generate_stories(model, tokenizer, prompts, do_sample=True, min_length=100, max_new_tokens=1000):
#     generated_texts = []
#     for prompts_batch in tqdm(batch(prompts, 3)):
#         inputs = tokenizer(prompts_batch, return_tensors="pt", padding=True).to("cuda")
#         with torch.no_grad():
#             generated_ids = model.generate(**inputs, do_sample=do_sample, min_length=min_length, max_new_tokens=max_new_tokens)
#         generated_texts.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))
#     return generated_texts

## Read Data

In [8]:
df_zs = pd.read_parquet('../data/generated_prompts/zeroshot.parquet')
df_zs_with_idea = pd.read_parquet('../data/generated_prompts/zeroshot_with_idea.parquet')

df_os = pd.read_parquet('../data/generated_prompts/oneshot.parquet')
df_os_with_idea = pd.read_parquet('../data/generated_prompts/oneshot_with_idea.parquet')

df_fs = pd.read_parquet('../data/generated_prompts/fewshot.parquet')
df_fs_with_idea = pd.read_parquet('../data/generated_prompts/fewshot_with_idea.parquet')

In [9]:
# sample
df_zs = df_zs.head(20)
df_zs_with_idea = df_zs_with_idea.head(20)

df_os = df_os.head(20)
df_os_with_idea = df_os_with_idea.head(20)

df_fs = df_fs.head(20)
df_fs_with_idea = df_fs_with_idea.head(20)

In [10]:
df_zs.head(1)
df_zs_with_idea.head(1)

df_os.head(1)
df_os_with_idea.head(1)

df_fs.head(1)
df_fs_with_idea.head(1)

,wp_prompt
0,Generate story using following story ouline: \...


,wp_prompt
0,Generate story using following story idea and ...


,wp_prompt
0,Generate story using following story ouline: \...


,wp_prompt
0,Generate story using following story idea and ...


,wp_prompt
0,Generate story using following story ouline: \...


,wp_prompt
0,Generate story using following story idea and ...


In [11]:
df_zs_with_idea['wp_prompt'].apply(len)

0     1719
1     1346
2     1260
3     1098
4     1840
5      685
6     1345
7      786
8     2240
9     1014
10    1480
12    1514
13    1116
14    1538
15    1072
18     989
19    1007
20    1730
21    1423
23    2475
Name: wp_prompt, dtype: int64

In [14]:
print(df_zs_with_idea['wp_prompt'][0])

Generate story using following story idea and story ouline: 

IDEA: 
You 've finally managed to discover the secret to immortality . Suddenly , Death appears before you , hands you a business card , and says , `` When you realize living forever sucks , call this number , I 've got a job offer for you . '' 

OUTLINE: 
At first I was scared , each time I could feel my city , my current generation collapse , break into the black hole that thrives within it , I could feel humanity , the way I 'm able to feel my body..
After a few hundred years , the pattern became obvious , no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly .
5 world wars have passed , and now they feel like a simple sickeness that would pass by every so often , I could no longer evaluate the individual human as a being of its own , the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much mor

## Generate Stories

In [15]:
# list(df_zs_with_idea['wp_prompt'])

In [16]:
# inputs = tokenizer_bloomz_1b7(list(df_zs_with_idea['wp_prompt'])[:2], return_tensors="pt", padding=True).to("cuda")
# with torch.no_grad():
#     generated_ids = model_bloomz_1b7.generate(**inputs, do_sample=False, min_length=100, max_new_tokens=1000)
# generated_texts = tokenizer_bloomz_1b7.batch_decode(generated_ids, skip_special_tokens=True)

In [17]:
# print(generated_texts[0])

In [27]:
story = generate_stories(
    model_bloomz_1b7, 
    tokenizer_bloomz_1b7, 
    [df_zs['wp_prompt'][0]], 
    do_sample=True, min_length=500, max_new_tokens=500
)
print(story[0])

  0%|          | 0/1 [00:00<?, ?it/s]

Generate story using following story ouline: 

OUTLINE: 
At first I was scared, each time I could feel my city, my current generation collapse, break into the black hole that thrives within it, I could feel humanity, the way I'm able to feel my body..
After a few hundred years, the pattern became obvious, no longer the war and damage that would devastate me over and over again in the far past was effecting me so dominantly.
5 world wars have passed, and now they feel like a simple sickeness that would pass by every so often, I could no longer evaluate the individual human as a being of its own, the importance of mortals is merely the same as the importance of my skin cells ; They are a part of a mechanism so much more advanced, a mechanism that is so dear to my fallen heart a mechanism that I have seen fall and rise so many times, a mechanism that when lost all of which it had, had me loosing my will to live, for the first time in all of my thousands years of existence.
a skill that ha

In [16]:
df_zs_with_idea['generated_text'] = generate_stories(
    model_bloomz_1b7, 
    tokenizer_bloomz_1b7, 
    df_zs_with_idea['wp_prompt'].tolist(), 
    do_sample=True, min_length=100, max_new_tokens=500)

  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
df_os_with_idea['generated_text'] = generate_stories(
    model_bloomz_1b7, 
    tokenizer_bloomz_1b7, 
    df_os_with_idea['wp_prompt'].tolist(), 
    do_sample=True, min_length=100, max_new_tokens=500)

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
df_fs_with_idea['generated_text'] = generate_stories(
    model_bloomz_1b7, 
    tokenizer_bloomz_1b7, 
    df_fs_with_idea['wp_prompt'].tolist(), 
    do_sample=True, min_length=100, max_new_tokens=500)

In [ ]:
df_zs_with_idea['generated_text'][0]